In [1]:
import pandas as pd 

data_row=pd.read_csv("/Users/withmocha/Desktop/DATA/BOAZ/미니 프로젝트 1/data/test data/news.csv",index_col=False)

data_row

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [2]:
data_row.drop(columns=[data_row.columns[0]],axis=1,inplace=True)

In [3]:
data_row=data_row.dropna(axis=0)

In [4]:
text_data=data_row.drop(columns=['label'])
label=pd.DataFrame(data_row['label'])
print(text_data.shape)
print(label.shape)

(6335, 2)
(6335, 1)


In [5]:
for i in range(label.shape[0]):
    if label.iloc[i,0]=='FAKE':
        label.iloc[i,0]=0
    else:
        label.iloc[i,0]=1

In [6]:
label

,label
0,0
1,0
2,1
3,0
4,1
...,...
6330,1
6331,0
6332,0
6333,1


In [7]:
text_data

,title,text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...
...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene..."


In [8]:
vocab_size=10000
messages=text_data.copy()

In [9]:
messages.reset_index(inplace=True)
messages

,index,title,text
0,0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...
...,...,...,...
6330,6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...
6331,6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...
6333,6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene..."


In [10]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/withmocha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    
  
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [12]:
max_length = max(len(sentence.split()) for sentence in corpus)

print("max_length:", max_length)

max_length: 26


In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import Dense, Dropout

onehot_repr=[one_hot(words,vocab_size) for words in corpus] 

In [14]:
sentence_length = 50
embedding_vector_features=100
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sentence_length)
print(embedded_docs)

[[   0    0    0 ... 9724 5499 4442]
 [   0    0    0 ... 2624 1322 4424]
 [   0    0    0 ... 4168 5829 6271]
 ...
 [   0    0    0 ... 8175 9092 1460]
 [   0    0    0 ... 5138 5816 2741]
 [   0    0    0 ... 4776 2624 5625]]


In [15]:
import numpy as np  

X_final = np.array(embedded_docs)
y_final = label
X_final.shape,y_final.shape

((6335, 50), (6335, 1))

In [16]:
import tensorflow as tf 
from keras.models import load_model

lstm_model_path='/Users/withmocha/Desktop/DATA/BOAZ/미니 프로젝트 1/model/lstm/LSTM model.keras'
lstm_transformer_path='/Users/withmocha/Desktop/DATA/BOAZ/미니 프로젝트 1/model/lstm transformer/LSTM-Transformer model.keras'
transformer_path='/Users/withmocha/Desktop/DATA/BOAZ/미니 프로젝트 1/model/transformer/Transformer model.keras'
BILSTM_path='/Users/withmocha/Desktop/DATA/BOAZ/미니 프로젝트 1/model/bi lstm/Bi-LSTM model.keras'

# load_model 함수 호출 시 custom_objects 인자 추가
lstm_model = tf.keras.models.load_model(lstm_model_path)
lstm_transformer_model = tf.keras.models.load_model(lstm_transformer_path)
transformer_model = tf.keras.models.load_model(transformer_path)
BILSTM_model = tf.keras.models.load_model(BILSTM_path)

In [17]:
lstm_prob = lstm_model.predict(X_final).ravel()
lstm_prob

198/198 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step


array([9.9566323e-01, 9.6726304e-01, 4.7511174e-04, ..., 9.6498615e-01,
       3.4465134e-01, 7.0601070e-01], dtype=float32)

In [18]:
y_pred = (lstm_prob > 0.5).astype("int32")

In [19]:
y_pred


array([1, 1, 0, ..., 1, 0, 1], dtype=int32)

In [20]:
y_final=np.array(label['label'])

In [21]:
y_final

array([0, 0, 1, ..., 0, 1, 1], dtype=object)

In [22]:
y_final=y_final.astype('int32')

In [23]:
from sklearn.metrics import accuracy_score

LSTM_accuracy = accuracy_score(y_final, y_pred)
print(f'LSTM Accuracy: {LSTM_accuracy:.2f}')

LSTM Accuracy: 0.51


In [24]:
y_final

array([0, 0, 1, ..., 0, 1, 1], dtype=int32)

In [25]:
lstm_transformer_prob = lstm_transformer_model.predict(X_final).ravel()
lstm_transformer_prob

198/198 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step


array([0.9602257 , 0.71938336, 0.57057947, ..., 0.07609513, 0.8082561 ,
       0.73416704], dtype=float32)

In [26]:
y_pred = (lstm_prob > 0.5).astype("int32")

In [27]:
lstm_transformer_prob[:10]

array([0.9602257 , 0.71938336, 0.57057947, 0.7164875 , 0.78628474,
       0.96531874, 0.16365188, 0.9563427 , 0.42577168, 0.26231563],
      dtype=float32)

In [28]:
LSTM_Transformer_accuracy = accuracy_score(y_final, y_pred)
print(f'LSTM transformer Accuracy: {LSTM_Transformer_accuracy:.2f}')

LSTM transformer Accuracy: 0.51


In [29]:
transformer_prob = transformer_model.predict(X_final).ravel()
transformer_prob

198/198 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step


array([0.9998776 , 0.99609065, 0.00604032, ..., 0.41209978, 0.9999956 ,
       0.99999404], dtype=float32)

In [30]:
y_pred = (transformer_prob > 0.5).astype("int32")

In [31]:
Transformer_accuracy = accuracy_score(y_final, y_pred)
print(f'Transformer Accuracy: {Transformer_accuracy:.2f}')

Transformer Accuracy: 0.50


In [32]:
BILSTM_prob = BILSTM_model.predict(X_final).ravel()
BILSTM_prob

198/198 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step


array([0.05351206, 0.98833925, 0.97087014, ..., 0.9251729 , 0.01361466,
       0.01419893], dtype=float32)

In [33]:
y_pred = (BILSTM_prob > 0.5).astype("int32")

In [34]:
BILSTM_accuracy = accuracy_score(y_final, y_pred)
print(f'BILSTM Accuracy: {BILSTM_accuracy:.2f}')

BILSTM Accuracy: 0.47


In [35]:
print(f'LSTM Accuracy: {LSTM_accuracy:.2f}')
print(f'LSTM transformer Accuracy: {LSTM_Transformer_accuracy:.2f}')
print(f'Transformer Accuracy: {Transformer_accuracy:.2f}')
print(f'BILSTM Accuracy: {BILSTM_accuracy:.2f}')

LSTM Accuracy: 0.51
LSTM transformer Accuracy: 0.51
Transformer Accuracy: 0.50
BILSTM Accuracy: 0.47
